# Run common functions

In [0]:
%run "/Workspace/tokyo-olympic/common_functions/common_functions"

# Config for mounting

In [0]:
client_id = dbutils.secrets.get(scope='tokyo-olympic',key='clientid')
tenant_id = dbutils.secrets.get(scope='tokyo-olympic',key='tenantid')
secret_key = dbutils.secrets.get(scope='tokyo-olympic',key='secretkey')

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": secret_key,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}

In [0]:
# dbutils.fs.mount(
#     source = "abfss://tokyo-olympic-data@tidetokyoolympic.dfs.core.windows.net",
#     mount_point = "/mnt/tokyo-olympic",
#     extra_configs = configs
# )

In [0]:
%fs
ls "/mnt/tokyo-olympic"

path,name,size,modificationTime
dbfs:/mnt/tokyo-olympic/presentation-data/,presentation-data/,0,1726048654000
dbfs:/mnt/tokyo-olympic/raw-data/,raw-data/,0,1726040138000
dbfs:/mnt/tokyo-olympic/transformed-data/,transformed-data/,0,1726040149000


# Read from raw-data

In [0]:
athletes = read_raw_csv_file("athletes")
coaches = read_raw_csv_file("coaches")
entriesgender = read_raw_csv_file("entriesgender")
medals = read_raw_csv_file("medals")
teams = read_raw_csv_file("teams")

In [0]:
athletes.show(truncate=False)

+-----------------------+------------------------+-------------------+
|PersonName             |Country                 |Discipline         |
+-----------------------+------------------------+-------------------+
|AALERUD Katrine        |Norway                  |Cycling Road       |
|ABAD Nestor            |Spain                   |Artistic Gymnastics|
|ABAGNALE Giovanni      |Italy                   |Rowing             |
|ABALDE Alberto         |Spain                   |Basketball         |
|ABALDE Tamara          |Spain                   |Basketball         |
|ABALO Luc              |France                  |Handball           |
|ABAROA Cesar           |Chile                   |Rowing             |
|ABASS Abobakr          |Sudan                   |Swimming           |
|ABBASALI Hamideh       |Islamic Republic of Iran|Karate             |
|ABBASOV Islam          |Azerbaijan              |Wrestling          |
|ABBINGH Lois           |Netherlands             |Handball           |
|ABBOT

# Find top 5 countries with the highest number of gold medals

In [0]:
# Find top 5 countries with the highest number of gold medals
top_5_countries_by_gold_medals = medals.select("TeamCountry","Gold").orderBy("Gold", ascending=False).limit(5)

In [0]:
top_5_countries_by_gold_medals.show(truncate=False)

+--------------------------+----+
|TeamCountry               |Gold|
+--------------------------+----+
|United States of America  |39  |
|People's Republic of China|38  |
|Japan                     |27  |
|Great Britain             |22  |
|ROC                       |20  |
+--------------------------+----+



# Calculate the average number of entries by gender for each discipline

In [0]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn(
                                                'avg_female', entriesgender['Female'] / entriesgender['Total']
                                                ).withColumn(
                                                    'avg_male', entriesgender['Male'] / entriesgender['Total']
                                                )

In [0]:
average_entries_by_gender.show(truncate=False)

+---------------------+------+----+-----+-------------------+-------------------+
|Discipline           |Female|Male|Total|avg_female         |avg_male           |
+---------------------+------+----+-----+-------------------+-------------------+
|3x3 Basketball       |32    |32  |64   |0.5                |0.5                |
|Archery              |64    |64  |128  |0.5                |0.5                |
|Artistic Gymnastics  |98    |98  |196  |0.5                |0.5                |
|Artistic Swimming    |105   |0   |105  |1.0                |0.0                |
|Athletics            |969   |1072|2041 |0.4747672709456149 |0.5252327290543851 |
|Badminton            |86    |87  |173  |0.49710982658959535|0.5028901734104047 |
|Baseball/Softball    |90    |144 |234  |0.38461538461538464|0.6153846153846154 |
|Basketball           |144   |144 |288  |0.5                |0.5                |
|Beach Volleyball     |48    |48  |96   |0.5                |0.5                |
|Boxing         

# Top 5 discipline has most female


In [0]:
top_5_discipline_most_female = entriesgender.select('Discipline','Female') \
                                      .orderBy('Female', ascending=False) \
                                      .limit(5)

In [0]:
top_5_discipline_most_female.show(truncate=False)

+----------+------+
|Discipline|Female|
+----------+------+
|Athletics |969   |
|Swimming  |361   |
|Football  |264   |
|Rowing    |257   |
|Hockey    |192   |
+----------+------+



# Top 5 discipline has most male

In [0]:
top_5_discipline_most_male = entriesgender.select('Discipline','Male') \
                                      .orderBy('Male', ascending=False) \
                                      .limit(5)

In [0]:
top_5_discipline_most_male.show(truncate=False)

+----------+----+
|Discipline|Male|
+----------+----+
|Athletics |1072|
|Swimming  |418 |
|Football  |344 |
|Rowing    |265 |
|Judo      |201 |
+----------+----+



# Find country has most players by discipline

In [0]:
from pyspark.sql.functions import col,row_number
from pyspark.sql.window import Window

window_by_discipline = Window.partitionBy('Discipline').orderBy(col('number_of_players').desc())
row_number_by_window = row_number().over(window_by_discipline)

country_most_players_by_discipline = athletes.select('Country','Discipline') \
                                             .groupBy(['Discipline','Country']).count().select(
                                                                                            col('Country'),
                                                                                            col('Discipline'),
                                                                                            col('count').alias('number_of_players')
                                                                                        ) \
                                             .withColumn('rank',row_number_by_window) \
                                             .filter(col('rank') == 1) \
                                             .orderBy(col('number_of_players').desc()) \
                                             .select(
                                                 col('Discipline'),
                                                 col('Country').alias('country_has_most_player'),
                                                 col('number_of_players')
                                                 )

In [0]:
country_most_players_by_discipline.show(truncate=False)

+-----------------+--------------------------+-----------------+
|Discipline       |country_has_most_player   |number_of_players|
+-----------------+--------------------------+-----------------+
|Athletics        |United States of America  |144              |
|Football         |Australia                 |43               |
|Rowing           |Great Britain             |39               |
|Swimming         |United States of America  |39               |
|Baseball/Softball|United States of America  |38               |
|Hockey           |South Africa              |35               |
|Rugby Sevens     |Japan                     |32               |
|Handball         |Norway                    |30               |
|Water Polo       |Spain                     |26               |
|Basketball       |Spain                     |24               |
|Fencing          |United States of America  |24               |
|Shooting         |People's Republic of China|24               |
|Volleyball       |Brazil

# Load to transformed-data

In [0]:
write_transformed_csv_file(athletes,"athletes")
write_transformed_csv_file(coaches,"coaches")
write_transformed_csv_file(entriesgender,"entriesgender")
write_transformed_csv_file(medals,"medals")
write_transformed_csv_file(teams,"teams")

# Load to presentation-data

In [0]:
write_presentation_csv_file(top_5_countries_by_gold_medals,"top_5_countries_by_gold_medals")
write_presentation_csv_file(average_entries_by_gender,"average_entries_by_gender")
write_presentation_csv_file(top_5_discipline_most_female,"top_5_discipline_most_female")
write_presentation_csv_file(top_5_discipline_most_male,"top_5_discipline_most_male")
write_presentation_csv_file(country_most_players_by_discipline,"country_most_players_by_discipline")